<a href="https://colab.research.google.com/github/annogass/ml-4/blob/main/simple_cnn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install wandb
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anigasitashvili","key":"31ff3d13351526141cdce5175f597602"}'}

In [3]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 81% 231M/285M [00:00<00:00, 645MB/s] 
100% 285M/285M [00:00<00:00, 517MB/s]


In [6]:
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip -d data/

In [7]:
!ls data

example_submission.csv	fer2013.tar.gz	icml_face_data.csv  test.csv  train.csv


In [8]:
!pip install -q wandb

In [9]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: agasi22 (agasi22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
import pandas as pd
import numpy as np

In [11]:
train_df_t = pd.read_csv('/content/data/icml_face_data.csv')

In [12]:
train_df_t.shape

(35887, 3)

In [13]:
train = train_df_t[train_df_t[' Usage'] == 'Training']
validation = train_df_t[train_df_t[' Usage'] == 'PrivateTest']
test = train_df_t[train_df_t[' Usage'] == 'PublicTest']

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Custom Dataset
class FERDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.pixels = df[' pixels'].tolist()
        self.emotions = df['emotion'].tolist()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        pixel_str = self.pixels[idx]
        pixels = np.array([int(p) for p in pixel_str.split()]).reshape(48, 48).astype(np.float32)
        pixels = pixels / 255.0  # Normalize
        pixels = torch.from_numpy(pixels).unsqueeze(0)  # Add channel dimension
        emotion = self.emotions[idx]
        return pixels, emotion

In [15]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 12 * 12)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [17]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd

def train_model(config=None):
    # Initialize W&B
    wandb.init(
        project="fer-challenge",
        config=config,
        name="simple-cnn-v1",  # You can change this per experiment
        notes="SimpleCNN baseline with W&B logging"
    )
    config = wandb.config

    # Load data
    df = pd.read_csv('/content/data/icml_face_data.csv')
    train_df = df[df[' Usage'] == 'Training']
    val_df = df[df[' Usage'] == 'PrivateTest']

    train_dataset = FERDataset(train_df)
    val_dataset = FERDataset(val_df)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size)

    # Model, loss, optimizer
    model = SimpleCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    # Log model gradients/weights to W&B (optional)
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Training loop
    for epoch in range(config.epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0

        for inputs, labels in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels.data)

        train_loss = train_loss / len(train_loader)
        train_acc = train_correct.double() / len(train_dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader)
        val_acc = val_correct.double() / len(val_dataset)

        # Log metrics to W&B
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc.item(),
            "val_loss": val_loss,
            "val_acc": val_acc.item()
        })

        print(f"Epoch {epoch+1}/{config.epochs}")
        print(f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # Save model locally and log to W&B
    model_path = "simple_cnn_model.pth"
    torch.save(model.state_dict(), model_path)
    wandb.save(model_path)

    # Finish W&B run
    wandb.finish()

# Configuration
config = {
    "learning_rate": 0.001,
    "batch_size": 64,
    "epochs": 15,
    "architecture": "SimpleCNN"
}

# Start training
train_model(config)


100%|██████████| 449/449 [01:45<00:00,  4.27it/s]


Epoch 1/15
Train Loss: 1.7341 Acc: 0.2957
Val Loss: 1.5916 Acc: 0.3773


100%|██████████| 449/449 [01:46<00:00,  4.20it/s]


Epoch 2/15
Train Loss: 1.5898 Acc: 0.3765
Val Loss: 1.5050 Acc: 0.4313


100%|██████████| 449/449 [01:49<00:00,  4.11it/s]


...

100%|██████████| 449/449 [01:43<00:00,  4.34it/s]


Epoch 14/15
Train Loss: 1.1054 Acc: 0.5693
Val Loss: 1.2321 Acc: 0.5297


100%|██████████| 449/449 [01:44<00:00,  4.31it/s]


Epoch 15/15
Train Loss: 1.0813 Acc: 0.5800
Val Loss: 1.2441 Acc: 0.5311


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▃▄▅▅▆▆▆▆▇▇▇███
train_loss,█▆▅▅▄▄▃▃▃▃▂▂▂▁▁
val_acc,▁▃▅▅▆▆▇▇▇▇█████
val_loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁
epoch,15
train_acc,0.57996
train_loss,1.08129
val_acc,0.53107
val_loss,1.2441
